# Домашняя работа 

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов 

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("ds_salaries.csv")

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

Ищем числовые данные для того, чтобы выполнить дальнейшие номера (без перевода с помощью one-hot-кодирования категориальных, тк оно требуется ниже), а также делаем проверку на пропуски: их нет

In [4]:
df.drop(columns='salary', inplace=True)

In [5]:
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [6]:
df.isnull().any()

work_year             False
experience_level      False
employment_type       False
job_title             False
salary_currency       False
salary_in_usd         False
employee_residence    False
remote_ratio          False
company_location      False
company_size          False
dtype: bool

Сразу избавились от salary

In [7]:
num = ['work_year', 'salary_in_usd', 'remote_ratio']

In [8]:
df_numerics = df[num]
df_numerics.head()

,work_year,salary_in_usd,remote_ratio
0,2023,85847,100
1,2023,30000,100
2,2023,25500,100
3,2023,175000,100
4,2023,120000,100


In [9]:
df_n = df_numerics.copy()
df_n['work_year'] = 2023 - df_n.work_year + 1
df_n.head()

,work_year,salary_in_usd,remote_ratio
0,1,85847,100
1,1,30000,100
2,1,25500,100
3,1,175000,100
4,1,120000,100


In [10]:
X = df_numerics.drop(columns=['salary_in_usd']).values
y = df_numerics['salary_in_usd'].values

In [11]:
X_n = df_n.drop(columns = ['salary_in_usd'])
y_n = df_n['salary_in_usd'].values

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [14]:
X_valid.shape

(376, 2)

In [15]:
X_test.shape

(375, 2)

In [16]:
X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, y_n, test_size = 0.1, random_state = 42)
X_n_train, X_n_valid, y_n_train, y_n_valid = train_test_split(X_n_train, y_n_train, test_size = 0.125, random_state = 42)

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
lg = LinearRegression()

In [19]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [20]:
lg.fit(X_train, y_train)
y_lg_pred = lg.predict(X_test)

In [21]:
print('MAPE:', mean_absolute_percentage_error(y_test, y_lg_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_lg_pred)))

MAPE: 0.8028552771954365
RMSE: 60506.320834088125


In [22]:
y_lg_valid = lg.predict(X_valid)
print('MAPE:', mean_absolute_percentage_error(y_valid, y_lg_valid))
print('RMSE:', np.sqrt(mean_squared_error(y_valid, y_lg_valid)))

MAPE: 0.610367307889143
RMSE: 62197.6495539488


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [23]:
import xgboost as xgb
#from xgboost.sklearn import XGBRegressor
from sklearn.metrics import roc_auc_score

In [24]:
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
d_test = xgb.DMatrix(X_test, label=y_test)

In [25]:
df_err = pd.DataFrame(columns = ['mape', 'rmse', 'eta', 'max_depth'])
df_err.head()

,mape,rmse,eta,max_depth


**Попробуем подогнать параметры)))**

In [26]:
q = 0
for i in range(5, 31, 5):
    for j in range(1, 6):
        params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': i / 100, 'max_depth': j,  'seed': 42}
        model = xgb.train(params, d_train)
        y_pred_xgb = model.predict(d_valid)
        mape = mean_absolute_percentage_error(y_valid, y_pred_xgb)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred_xgb))
        df_err.loc[q, 'mape'] = mape
        df_err.loc[q, 'rmse'] = rmse
        df_err.loc[q, 'eta'] = i / 100
        df_err.loc[q, 'max_depth'] = j
        q += 1

df_err

,mape,rmse,eta,max_depth
0,0.603923,104993.570374,0.05,1
1,0.597508,104612.367338,0.05,2
2,0.599226,104583.557527,0.05,3
3,0.599201,104564.644295,0.05,4
4,0.599201,104564.644295,0.05,5
5,0.505843,79792.567853,0.1,1
6,0.498451,79385.742863,0.1,2
7,0.500523,79313.527023,0.1,3
8,0.50231,79492.934117,0.1,4
9,0.50231,79492.934117,0.1,5


In [27]:
df_err[df_err.rmse == df_err.rmse.min()]

,mape,rmse,eta,max_depth
25,0.593277,62725.620177,0.3,1


In [28]:
df_err[df_err.mape == df_err.mape.min()]

,mape,rmse,eta,max_depth
6,0.498451,79385.742863,0.1,2


##### Однако если посмотреть по значениям, то наиболее оптимальными будут eta = 0.2, max_depth=5 (при них и mape, и rmse относительно малы)

In [29]:
params1 = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.2, 'max_depth': 5,  'seed': 42}

In [30]:
model_xgb = xgb.train(params1, d_train)
y_xgb_predict = model_xgb.predict(d_test)
print('MAPE:', mean_absolute_percentage_error(y_test, y_xgb_predict))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_xgb_predict)))

MAPE: 0.6793856591149885
RMSE: 59806.5418419663


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [31]:
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [32]:
cat = ['experience_level', 'employment_type', 'job_title', 
       'salary_currency', 'employee_residence', 'company_location', 'company_size']

In [33]:
df_opposites = pd.DataFrame(columns=['model', 'code', 'rmse', 'mape'])
df_opposites

,model,code,rmse,mape


In [34]:
#pip install category_encoders

In [35]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder

In [36]:
df_categorials = pd.get_dummies(df)

In [37]:
df_train, df_test_val = train_test_split(df_categorials, test_size=0.2, random_state=42)
df_test, df_val = train_test_split(df_test_val, test_size=0.5, random_state=42)

In [38]:
X_train_oh = df_train.drop(columns ='salary_in_usd')
y_train_oh = df_train['salary_in_usd']
X_test_oh = df_test.drop(columns = 'salary_in_usd')
y_test_oh = df_test['salary_in_usd']
X_valid_oh = df_val.drop(columns = 'salary_in_usd')
y_valid_oh = df_val['salary_in_usd']

In [39]:
lg = LinearRegression()
lg.fit(X_train_oh, y_train_oh)
y_test_pred_lg_oh = lg.predict(X_test_oh)

In [40]:
np.square(mean_squared_error(y_test_oh, y_test_pred_lg_oh))

2.6718879159579055e+54

In [41]:
df_opposites.loc[0, 'model'] = 'linear regression'
df_opposites.loc[0, 'code'] = 'one-hot'
df_opposites.loc[0, 'rmse'] = np.square(mean_squared_error(y_test_oh, y_test_pred_lg_oh))
df_opposites.loc[0, 'mape'] = mean_absolute_percentage_error(y_test_oh, y_test_pred_lg_oh)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326


In [42]:
dfle = df.copy()

In [43]:
labelEncoder = LabelEncoder()
dfle[cat] = dfle[cat].apply(labelEncoder.fit_transform)
X = dfle.drop(columns='salary_in_usd')
y = dfle['salary_in_usd']

In [44]:
dfle.head()

,work_year,experience_level,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,3,2,84,7,85847,26,100,25,0
1,2023,2,0,66,19,30000,75,100,70,2
2,2023,2,0,66,19,25500,75,100,70,2
3,2023,3,2,47,19,175000,11,100,12,1
4,2023,3,2,47,19,120000,11,100,12,1


In [45]:
X_train_el, X_test_el, y_train_el, y_test_el = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_el, X_valid_el, y_test_el, y_valid_el = train_test_split(X_test_el, y_test_el, test_size=0.5, random_state=42)

In [46]:
lg.fit(X_train_el, y_train_el)
y_test_pred_lg_el = lg.predict(X_test_el)


In [47]:
df_opposites.loc[1, 'model'] = 'linear regression'
df_opposites.loc[1, 'code'] = 'label encoder'
df_opposites.loc[1, 'rmse'] = np.square(mean_squared_error(y_test_el, y_test_pred_lg_el))
df_opposites.loc[1, 'mape'] = mean_absolute_percentage_error(y_test_el, y_test_pred_lg_el)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326
1,linear regression,label encoder,6422403073997485056.0,0.593067


In [48]:
dfte = df.copy()

In [49]:

X = dfte.drop(columns='salary_in_usd')
y = dfte['salary_in_usd']

In [50]:
encoder = TargetEncoder(cols=X.select_dtypes(include='object'))
X = encoder.fit_transform(X, y)

In [51]:
X_train_te, X_test_te, y_train_te, y_test_te = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_te, X_valid_te, y_test_te, y_valid_te = train_test_split(X_test_te, y_test_te, test_size=0.5, random_state=42)

In [52]:
lg.fit(X_train_te, y_train_te)
y_test_pred_lg_te = lg.predict(X_test_te)

In [53]:
df_opposites.loc[2, 'model'] = 'linear regression'
df_opposites.loc[2, 'code'] = 'mean target encoder'
df_opposites.loc[2, 'rmse'] = np.square(mean_squared_error(y_test_te, y_test_pred_lg_te))
df_opposites.loc[2, 'mape'] = mean_absolute_percentage_error(y_test_te, y_test_pred_lg_te)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326
1,linear regression,label encoder,6422403073997485056.0,0.593067
2,linear regression,mean target encoder,4164868437212072448.0,0.421401


Теперь наконец-то поработаем с бустингом

In [54]:
d_train_oh = xgb.DMatrix(X_train_oh, label=y_train_oh)
d_valid_oh = xgb.DMatrix(X_valid_oh, label=y_valid_oh)
d_test_oh = xgb.DMatrix(X_test_oh, label=y_test_oh)

In [55]:
df_err = pd.DataFrame(columns = ['mape', 'rmse', 'eta', 'max_depth'])
df_err.head()

,mape,rmse,eta,max_depth


In [56]:
q = 0
for i in range(5, 31, 5):
    for j in range(1, 6):
        params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': i / 100, 'max_depth': j,  'seed': 42}
        model = xgb.train(params, d_train_oh)
        y_pred_xgb = model.predict(d_valid_oh)
        mape = mean_absolute_percentage_error(y_valid_oh, y_pred_xgb)
        rmse = np.sqrt(mean_squared_error(y_valid_oh, y_pred_xgb))
        df_err.loc[q, 'mape'] = mape
        df_err.loc[q, 'rmse'] = rmse
        df_err.loc[q, 'eta'] = i / 100
        df_err.loc[q, 'max_depth'] = j
        q += 1

df_err

,mape,rmse,eta,max_depth
0,0.566082,102870.371436,0.05,1
1,0.559452,102061.866773,0.05,2
2,0.555251,101426.176717,0.05,3
3,0.558497,101302.575664,0.05,4
4,0.557604,100969.246047,0.05,5
5,0.404918,75652.666703,0.1,1
6,0.379288,73967.376211,0.1,2
7,0.362281,72996.83239,0.1,3
8,0.358672,72649.562695,0.1,4
9,0.350534,72119.575805,0.1,5


Самая крутая модель с eta = 0.3 и глубиной 5, проверим на тестовой выборке

In [57]:
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.3, 'max_depth': 5,  'seed': 42}
model = xgb.train(params, d_train_oh)
y_pred_xgb = model.predict(d_test_oh)
df_opposites.loc[3, 'model'] = 'xgboost'
df_opposites.loc[3, 'code'] = 'one-hot'
df_opposites.loc[3, 'rmse'] = np.square(mean_squared_error(y_test_oh, y_pred_xgb))
df_opposites.loc[3, 'mape'] = mean_absolute_percentage_error(y_test_oh, y_pred_xgb)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326
1,linear regression,label encoder,6422403073997485056.0,0.593067
2,linear regression,mean target encoder,4164868437212072448.0,0.421401
3,xgboost,one-hot,4596067502943561728.0,0.394038


In [58]:
d_train_el = xgb.DMatrix(X_train_el, label=y_train_el)
d_valid_el = xgb.DMatrix(X_valid_el, label=y_valid_el)
d_test_el = xgb.DMatrix(X_test_el, label=y_test_el)

In [59]:
df_err = pd.DataFrame(columns = ['mape', 'rmse', 'eta', 'max_depth'])
df_err.head()

,mape,rmse,eta,max_depth


In [60]:
q = 0
for i in range(5, 31, 5):
    for j in range(1, 6):
        params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': i / 100, 'max_depth': j,  'seed': 42}
        model = xgb.train(params, d_train_el)
        y_pred_xgb = model.predict(d_valid_el)
        mape = mean_absolute_percentage_error(y_valid_el, y_pred_xgb)
        rmse = np.sqrt(mean_squared_error(y_valid_el, y_pred_xgb))
        df_err.loc[q, 'mape'] = mape
        df_err.loc[q, 'rmse'] = rmse
        df_err.loc[q, 'eta'] = i / 100
        df_err.loc[q, 'max_depth'] = j
        q += 1

df_err

,mape,rmse,eta,max_depth
0,0.566339,102868.019757,0.05,1
1,0.55265,102182.227496,0.05,2
2,0.555983,101527.095447,0.05,3
3,0.558524,101248.440159,0.05,4
4,0.561799,101234.780039,0.05,5
5,0.404398,75662.591822,0.1,1
6,0.373204,74294.069736,0.1,2
7,0.363336,73218.695959,0.1,3
8,0.360121,72608.03837,0.1,4
9,0.358457,72446.108199,0.1,5


In [61]:
df_err[df_err.rmse == df_err.rmse.min()]

,mape,rmse,eta,max_depth
29,0.325147,50242.412832,0.3,5


In [62]:
df_err[df_err.mape == df_err.mape.min()]

,mape,rmse,eta,max_depth
19,0.305592,53019.9148,0.2,5


Самая хорошая модель по двум характеристикам - с eta = 0.25 и глубиной 5

In [63]:
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.25, 'max_depth': 5,  'seed': 42}
model = xgb.train(params, d_train_el)
y_pred_xgb_el = model.predict(d_test_el)
df_opposites.loc[4, 'model'] = 'xgboost'
df_opposites.loc[4, 'code'] = 'label encoder'
df_opposites.loc[4, 'rmse'] = np.square(mean_squared_error(y_test_el, y_pred_xgb_el))
df_opposites.loc[4, 'mape'] = mean_absolute_percentage_error(y_test_el, y_pred_xgb_el)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326
1,linear regression,label encoder,6422403073997485056.0,0.593067
2,linear regression,mean target encoder,4164868437212072448.0,0.421401
3,xgboost,one-hot,4596067502943561728.0,0.394038
4,xgboost,label encoder,4783351004951310336.0,0.396177


In [64]:
d_train_te = xgb.DMatrix(X_train_te, label=y_train_te)
d_valid_te = xgb.DMatrix(X_valid_te, label=y_valid_te)
d_test_te = xgb.DMatrix(X_test_te, label=y_test_te)

In [65]:
df_err = pd.DataFrame(columns = ['mape', 'rmse', 'eta', 'max_depth'])
df_err.head()

,mape,rmse,eta,max_depth


In [66]:
q = 0
for i in range(5, 31, 5):
    for j in range(1, 6):
        params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': i / 100, 'max_depth': j,  'seed': 42}
        model = xgb.train(params, d_train_te)
        y_pred_xgb = model.predict(d_valid_te)
        mape = mean_absolute_percentage_error(y_valid_te, y_pred_xgb)
        rmse = np.sqrt(mean_squared_error(y_valid_te, y_pred_xgb))
        df_err.loc[q, 'mape'] = mape
        df_err.loc[q, 'rmse'] = rmse
        df_err.loc[q, 'eta'] = i / 100
        df_err.loc[q, 'max_depth'] = j
        q += 1

df_err

,mape,rmse,eta,max_depth
0,0.555887,102107.909692,0.05,1
1,0.554683,101456.428683,0.05,2
2,0.559132,101060.964078,0.05,3
3,0.564149,100844.788448,0.05,4
4,0.564441,100778.423008,0.05,5
5,0.391013,74412.900421,0.1,1
6,0.36651,72923.30827,0.1,2
7,0.355153,72327.817077,0.1,3
8,0.351576,71949.303751,0.1,4
9,0.347504,71679.443999,0.1,5


In [67]:
df_err[df_err.rmse == df_err.rmse.min()]

,mape,rmse,eta,max_depth
29,0.306681,49017.280222,0.3,5


In [68]:
df_err[df_err.mape == df_err.mape.min()]

,mape,rmse,eta,max_depth
19,0.285828,51819.125124,0.2,5


Лучшие параметры как и в прошлом

In [69]:
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.25, 'max_depth': 5,  'seed': 42}
model = xgb.train(params, d_train_te)
y_pred_xgb_mne = model.predict(d_test_te)
df_opposites.loc[5, 'model'] = 'xgboost'
df_opposites.loc[5, 'code'] = 'mean target encoder'
df_opposites.loc[5, 'rmse'] = np.square(mean_squared_error(y_test_te, y_pred_xgb_mne))
df_opposites.loc[5, 'mape'] = mean_absolute_percentage_error(y_test_te, y_pred_xgb_mne)
df_opposites

,model,code,rmse,mape
0,linear regression,one-hot,2671887915957905492285175958741265734304143400...,251516042.002326
1,linear regression,label encoder,6422403073997485056.0,0.593067
2,linear regression,mean target encoder,4164868437212072448.0,0.421401
3,xgboost,one-hot,4596067502943561728.0,0.394038
4,xgboost,label encoder,4783351004951310336.0,0.396177
5,xgboost,mean target encoder,4258149796795272192.0,0.343751


Лучше всего работает xgboost с mean target encoder, а еще ошибка rmse огромная( Возможно, стоило отмасштабировать признаки

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [70]:
errors = np.abs(y_test_te - y_pred_xgb_mne)
error_df = pd.DataFrame({'Index': X_test_te.index, 'Error': errors})
top_20_errors = pd.DataFrame(error_df.nlargest(20, 'Error'))
top_20_errors

,Index,Error
1105,1105,213961.968750
478,478,172305.296875
358,358,137960.859375
2009,2009,136700.984375
3477,3477,123331.265625
2281,2281,123292.453125
3387,3387,122816.421875
549,549,119261.968750
2589,2589,114759.601562
3609,3609,111347.140625


In [71]:
err = top_20_errors.join(X_test_te, on='Index', how='inner')
err

,Index,Error,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
1105,1105,213961.968750,2023,153051.071542,138314.19957,140869.765476,149366.906638,152822.011651,0,151822.009539,143130.548367
478,478,172305.296875,2023,194926.185120,138314.19957,154211.124538,149366.906638,152822.011651,0,151822.009539,143130.548367
358,358,137960.859375,2023,153051.071542,138314.19957,154690.726644,149366.906638,152822.011651,100,151822.009539,143130.548367
2009,2009,136700.984375,2022,153051.071542,138314.19957,154690.726644,149366.906638,125409.049786,100,126699.968609,143130.548367
3477,3477,123331.265625,2022,78546.284375,138314.19957,126647.517548,149366.906638,120345.915710,100,117182.530481,78226.846266
2281,2281,123292.453125,2022,153051.071542,138314.19957,190103.568117,149366.906638,152822.011651,0,151822.009539,143130.548367
3387,3387,122816.421875,2022,153051.071542,138314.19957,142785.734615,149366.906638,152822.011651,100,151822.009539,143130.548367
549,549,119261.968750,2023,153051.071542,138314.19957,140869.765476,149366.906638,152822.011651,100,151822.009539,143130.548367
2589,2589,114759.601562,2022,153051.071542,138314.19957,120712.184130,149366.906638,36832.623336,100,120974.530086,118300.982379
3609,3609,111347.140625,2020,153051.071542,138314.19957,154131.272072,149366.906638,130281.781334,0,132932.975594,78226.846266


Во-первых, большинство ошибок у work_year = 2023, сам по себе этот признак немного странный)) Среди категориальных у всех ошибочных практически один и тот же уровень опыта (experience_level), один тип работы (employment_type), одна страна компании (company_location) и +- один размер компании (company_size)

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

Мне сразу не очень понравилось то, что в work_year стоит год (видимо начала работы) и я бы заменила его на разницу текущего года - 2023 и "старта" работы + 1, чтобы взять округленное вверх количество лет работы а также выкинули часть признаков.

In [72]:
dfte = df.copy()
dfte['work_year'] = 2023 - dfte.work_year + 1

In [73]:
X = dfte.drop(columns=['salary_in_usd',  'salary_currency', 'employee_residence', 'employment_type'])
y = dfte['salary_in_usd']

encoder = TargetEncoder(cols=X.select_dtypes(include='object'))
X = encoder.fit_transform(X, y)

In [75]:
X_train_te, X_test_te, y_train_te, y_test_te = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_te, X_valid_te, y_test_te, y_valid_te = train_test_split(X_test_te, y_test_te, test_size=0.5, random_state=42)

In [76]:
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'eta': 0.25, 'max_depth': 5,  'seed': 42}
model = xgb.train(params, d_train_te)
y_pred_xgb_mne = model.predict(d_test_te)
print('RMSE:', np.square(mean_squared_error(y_test_te, y_pred_xgb_mne)))
print('MAPE:', mean_absolute_percentage_error(y_test_te, y_pred_xgb_mne))

RMSE: 4.258149796795272e+18
MAPE: 0.34375104477908025


Ошибка не поменялась, выводы неверные(

P.S. Спасибо большое за оперативную проверку домашек🥺 Дальше номеров не будет(

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания